###Install dependencies

In [ ]:
!pip install -q -U transformers accelerate peft diffusers

###Setup pipeline

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(pipe_id, torch_dtype=torch.float16).to("cuda")

###Load toy LoRA weights

In [5]:
pipe.load_lora_weights("CiroN2022/toy-face", weight_name="toy_face_sdxl.safetensors", adapter_name="toy")

###Run inferece with toy LoRA

In [ ]:
prompt = "toy_face of a Jolly Roger holding a flower"

lora_scale = 0.9
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image

###Load LoRA pixel with adapter pixel

In [ ]:
pipe.load_lora_weights("nerijs/pixel-art-xl", weight_name="pixel-art-xl.safetensors", adapter_name="pixel")
pipe.set_adapters("pixel")

###Run inference with pixel LoRA and pixel adapter

In [ ]:
prompt = "a Jolly Roger holding a flower, pixel art"
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image

###Merge toy and pixel adapters

In [11]:
pipe.set_adapters(["pixel", "toy"], adapter_weights=[0.2, 0.9])

###Inference with merged adapters

In [ ]:
prompt = "toy_face of a Jolly Roger holding a flower, pixel art"
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": 1.0}, generator=torch.manual_seed(0)
).images[0]
image

###Managing of active adapters

In [ ]:
active_adapters = pipe.get_active_adapters()
active_adapters

list_adapters_component_wise = pipe.get_list_adapters()
list_adapters_component_wise

###Disable lora and run inference with initial SD

In [ ]:
pipe.disable_lora()

prompt = "toy_face of a Jolly Roger holding a flower"
image = pipe(prompt, num_inference_steps=30, generator=torch.manual_seed(0)).images[0]
image

###Back to lora adapters with more interesting settings

In [ ]:
pipe.enable_lora()
adapter_weight_scales_toy = 0.5
adapter_weight_scales_pixel = {
    # by default scale is 1.0
    "unet": {
        "down": 0.8,  # all transformers in the down-part will use scale 0.8
        "mid": 0.7,  # all transformers in the mid part will use the default scale 0.7
        "up": {
            "block_0": 0.5,  # all 3 transformers in the 0th block in the up-part will use scale 0.5
            "block_1": [0.3, 0.7, 0.9],  # the 3 transformers in the 1st block in the up-part will use scales 0.3, 0.7 and 0.9 respectively
            "block_2": 0.2 # all 3 transformers in the 2st block in the up-part will use scale 0.2
        }
    }
}
pipe.set_adapters(["toy", "pixel"], [adapter_weight_scales_toy, adapter_weight_scales_pixel])
prompt = "toy_face of a Jolly Roger holding a flower, pixel art"
image = pipe(prompt, num_inference_steps=30, generator=torch.manual_seed(0)).images[0]
image
